# SageMaker Training with MLflow

<div class="alert alert-block alert-info">
⚠️ 이 노트북과 함께 작동하는 것으로 알려진 최신 SageMaker Distribution 이미지 버전은 <code>3.1.0</code>입니다. 다른 버전에서 문제가 발생하면 버전 <code>3.1.0</code>으로 다운그레이드하세요. <b>이를 위해서는 JupyterApp을 중지하고, SageMaker Distribution 이미지를 <code>3.1.0</code>으로 다운그레이드한 다음 변경 사항을 적용하기 위해 JupyterLabApp을 재시작해야 합니다</b>.</div>

<div class="alert alert-warning"> 이 노트북은 SageMaker Managed MLflow가 실행 중인 인스턴스를 필요로 합니다.</div>

이 실습에서는 실험 추적을 위해 SageMaker Managed MLflow를 사용하는 방법을 보여드립니다.
몇 가지 시나리오를 보여드리며, 데이터 사이언티스트가 SageMaker Managed 인프라를 사용하여 원격 작업을 실행하기 전에 알고리즘을 테스트하기 위해 로컬 훈련 함수나 훈련 스크립트를 먼저 개발하는 워크플로우를 모방할 것입니다.

## 환경 설정

필요한 라이브러리를 설치합니다. `mlflow` 버전 `2.22.1`을 사용합니다.
MLflow에 데이터를 기록하기 위해 [`sagemaker-mlflow`](https://github.com/aws/sagemaker-mlflow) 플러그인을 설치해야 합니다.
이 플러그인은 Amazon SageMaker로 전송되는 모든 요청에 Signature V4 헤더를 생성하여 MLflow 기능을 사용하고, 추적 서버에 연결할 기능의 URL을 결정하며, SageMaker Model Registry에 모델을 등록합니다.
이 플러그인은 AWS IAM을 사용하여 인증 및 권한 부여를 수행하는 서비스가 사용할 SigV4 알고리즘으로 토큰을 생성합니다.

이 플러그인을 사용하면 `mlflow` 클라이언트 SDK를 추가 수정 없이 투명하게 사용하여 메타데이터와 아티팩트를 SageMaker Managed MLflow에 안전하게 기록할 수 있습니다.

In [ ]:
!pip install -q sagemaker[local] mlflow==2.22.1 sagemaker-mlflow

필요한 라이브러리를 가져오고 클라이언트 SDK를 초기화합니다

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.estimator import SKLearn

import boto3
import numpy as np
import pandas as pd
import os
import json

from IPython.display import Javascript, HTML

# Define session, role, and region so we can
# perform any SageMaker tasks we need
boto_session = boto3.Session()
sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_region_name
sm_client = boto_session.client("sagemaker")

sagemaker.__version__

이제 우리가 작업 중인 SageMaker Domain과 Space에 대한 정보를 추출합니다.
이 정보는 나중에 어떤 `user-profile`이 특정 MLflow 실행을 기록했는지에 대한 더 정확한 정보를 기록하는 데 유용할 것입니다.

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')

if not space_name:
    raise Exception(f"Cannot find the current domain. Make sure you run this notebook in a JupyterLab in the SageMaker AI Studio")
else:
    print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

print(f"User profile name: {user_profile_name}")

로컬 개발을 위해 우리가 사용 중인 SageMaker Distribution 이미지에 대한 참조를 갖는 것이 유용합니다.
아래와 같이 이 정보를 찾을 수 있습니다

In [ ]:
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
resource_spec = r['SpaceSettings']['JupyterLabAppSettings']['DefaultResourceSpec']
sm_image = resource_spec.get('SageMakerImageArn', 'not defined')
sm_image_version = resource_spec.get('SageMakerImageVersionAlias', 'not defined')
print(f"""
SageMaker image: \033[1m{sm_image}\033[0m
SageMaker image version: \033[1m{sm_image_version}\033[0m
""")


우리가 작업 중인 공간에 대한 전체 세부 정보는 다음과 같이 확인할 수 있습니다

In [ ]:
import pprint

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(r)

## MLflow 추적 서버

AWS 주도 이벤트에서 실행 중인 경우, MLflow 추적 서버가 이미 제공되었습니다.
또는 MLflow 추적 서버를 포함한 필요한 인프라를 생성하기 위해 CloudFormation 템플릿을 실행했는지 확인하세요.

In [ ]:
# Find an active MLflow server in the account
tracking_servers = [s['TrackingServerArn'] for s 
                    in sm_client.list_mlflow_tracking_servers()['TrackingServerSummaries']
                    if s['IsActive'] == 'Active']

if len(tracking_servers) < 1:
    print("You don't have any active MLflow servers. Trying to find a server in the status 'Creating'...")

    r = sm_client.list_mlflow_tracking_servers(
        TrackingServerStatus='Creating',
    )['TrackingServerSummaries']

    if len(r) < 1:
        print("You don't have any MLflow server in the status 'Creating'. Run the next code cell to create a new one.")
        mlflow_server_arn = None
        mlflow_name = None
    else:
        mlflow_server_arn = r[0]['TrackingServerArn']
        mlflow_name = r[0]['TrackingServerName']
        print(f"You have an MLflow server {mlflow_server_arn} in the status 'Creating', going to use this one")
else:
    mlflow_server_arn = tracking_servers[0]
    mlflow_name = tracking_servers[0].split('/')[1]
    print(f"You have {len(tracking_servers)} running MLflow server(s). Get the first server ARN:{mlflow_server_arn}")

mlflow_experiment_name = "sm-immersion-day-experiment"

## 데이터 준비

Iris 데이터셋을 다운로드하여 `./data` 폴더에 저장해 보겠습니다

In [ ]:
os.makedirs("./data", exist_ok=True)

s3_client = boto3.client("s3")
s3_client.download_file(
    f"sagemaker-example-files-prod-{region}", "datasets/tabular/iris/iris.data", "./data/iris.csv"
)

df_iris = pd.read_csv("./data/iris.csv", header=None)
df_iris[4] = df_iris[4].map({"Iris-setosa": 0, "Iris-versicolor": 1, "Iris-virginica": 2})
iris = df_iris[[4, 0, 1, 2, 3]].to_numpy()
np.savetxt("./data/iris.csv", iris, delimiter=",", fmt="%1.1f, %1.3f, %1.3f, %1.3f, %1.3f")


In [ ]:
# S3 prefix for the training dataset to be uploaded to
prefix = "DEMO-scikit-iris"

WORK_DIRECTORY = "data"

train_input = sagemaker_session.upload_data(
    WORK_DIRECTORY, key_prefix="{}/{}".format(prefix, WORK_DIRECTORY)
)

훈련 코드를 위한 폴더 준비

In [ ]:
!mkdir -p training_code

훈련 데이터를 Pandas DataFrame에 저장

In [ ]:
train_data = pd.read_csv('./data/iris.csv', header=None, engine="python")

## 원격 함수 실행

먼저 훈련 함수를 로컬에서 실행합니다. 함수 정의 상단에 주석 처리된 `@remote` 데코레이터를 확인하세요. 아래와 같이 정의된 경우, 이것은 로컬 런타임 환경에서 실행할 수 있는 일반 파이썬 함수일 뿐입니다.
`MLFLOW_TRACKING_URI` 환경 변수를 `mlflow_server_arn`으로 설정하여 클라이언트가 원격 MLflow 추적 서버에 로그를 기록하도록 합니다.
반면에 `LOGNAME` 환경 변수를 설정하면 실행을 기록하는 사용자를 더 쉽게 식별할 수 있습니다.

In [ ]:
os.environ['MLFLOW_TRACKING_URI'] = mlflow_server_arn
os.environ["LOGNAME"] = user_profile_name
os.environ["MLFLOW_EXPERIMENT_NAME"] = mlflow_experiment_name

# define a local function
# @remote
def train(train_data, max_leaf_nodes, run_name='Training-local-function-execution'):
    import mlflow
    from mlflow.models import infer_signature
    from sklearn import tree
    import pandas as pd

    # Enable autologging in MLflow for SKlearn
    mlflow.sklearn.autolog()

    with mlflow.start_run(run_name=run_name) as run:
        # labels are in the first column
        train_y = train_data.iloc[:, 0]
        train_X = train_data.iloc[:, 1:]

        # Here we support a single hyperparameter, 'max_leaf_nodes'. Note that you can add as many
        # as your training my require in the ArgumentParser above.

        # Now use scikit-learn's decision tree classifier to train the model.
        clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
        clf = clf.fit(train_X, train_y)

        predictions = clf.predict(train_X)
        signature = infer_signature(train_X, predictions)

        mlflow.set_tags(
            {
                'mlflow.source.name': "def train(...)",
                'mlflow.source.type': 'LOCAL',
            }
        )

        mlflow.sklearn.log_model(clf, "model", signature=signature)

In [ ]:
train(train_data, 5)

이제 SageMaker AI Managed MLflow에 기록된 실행 세부 정보를 확인해 보겠습니다.

In [ ]:
import mlflow

experiment_id = mlflow.get_experiment_by_name(mlflow_experiment_name).experiment_id
# get the last run in MLflow
last_run_id = mlflow.search_runs(
    experiment_ids=[experiment_id], 
    max_results=1, 
    order_by=["attributes.start_time DESC"]
)['run_id'][0]

# get the presigned url to open the MLflow UI
presigned_url = sm_client.create_presigned_mlflow_tracking_server_url(
    TrackingServerName=mlflow_name,
    ExpiresInSeconds=60,
    SessionExpirationDurationInSeconds=1800
)['AuthorizedUrl']

mlflow_run_link = f"{presigned_url.split('/auth')[0]}/#/experiments/{experiment_id}/runs/{last_run_id}"

먼저 사전 서명된 URL을 열어야 합니다

In [ ]:
# first open the MLflow UI - you can close a new opened window
display(Javascript('window.open("{}");'.format(presigned_url)))

그런 다음 마지막으로 기록된 실행의 세부 정보를 열 수 있습니다

In [ ]:
display(Javascript('window.open("{}");'.format(mlflow_run_link)))

이제 이 함수를 관리형 인프라에서 SageMaker Training 작업으로 실행할 준비를 해보겠습니다.
먼저 `requirements.txt` 파일에 의존성을 정의합니다.

In [ ]:
%%writefile training_code/requirements.txt
mlflow==2.22.1
sagemaker-mlflow==0.1.0

그런 다음 훈련 작업에 원하는 구성을 담은 `config.yml` 파일을 준비합니다.
`@remote` 데코레이터에 대해 구성할 수 있는 옵션에 대한 자세한 내용은 [공식 문서](https://docs.aws.amazon.com/sagemaker/latest/dg/train-remote-decorator-config.html)에서 확인할 수 있습니다.
이 경우, 추적 서버 URI를 항상 설정하지 않아도 되도록 `MLFLOW_TRACKING_URI` 환경 변수를 전달하고, 누가 무엇을 생성했는지 추적하기 위해 `LOGNAME`으로 `user_profile_name`을 전달하는 방법에 주목하세요.

In [ ]:
config_yaml = f"""
SchemaVersion: '1.0'
SageMaker:
  PythonSDK:
    Modules:
      TelemetryOptOut: true
      RemoteFunction:
        # role arn is not required if in SageMaker Notebook instance or SageMaker Studio
        # Uncomment the following line and replace with the right execution role if in a local IDE
        # RoleArn: <replace the role arn here>
        InstanceType: ml.m5.xlarge
        EnvironmentVariables: {{'MLFLOW_TRACKING_URI': {mlflow_server_arn}, 'LOGNAME': {user_profile_name}, 'MLFLOW_EXPERIMENT_NAME': {mlflow_experiment_name}}}
        Dependencies: ./training_code/requirements.txt
        IncludeLocalWorkDir: false
        CustomFileFilter:
          IgnoreNamePatterns:
          - "data/*"
          - "models/*"
          - "*.ipynb"
          - "__pycache__"

"""

print(config_yaml, file=open('config.yaml', 'w'))
print(config_yaml)

In [ ]:
import os

# Use the current working directory as the location for SageMaker Python SDK config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

In [ ]:
from sagemaker.remote_function import remote

# define a local function
@remote
def train(train_data, max_leaf_nodes, run_name='Training-remote-function-execution'):
    import mlflow
    from mlflow.models import infer_signature
    from sklearn import tree
    import pandas as pd

    # Enable autologging in MLflow for SKlearn
    mlflow.sklearn.autolog()

    with mlflow.start_run(run_name=run_name) as run:
        # labels are in the first column
        train_y = train_data.iloc[:, 0]
        train_X = train_data.iloc[:, 1:]

        # Now use scikit-learn's decision tree classifier to train the model.
        clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
        clf = clf.fit(train_X, train_y)

        predictions = clf.predict(train_X)
        signature = infer_signature(train_X, predictions)

        mlflow.set_tags(
            {
                'mlflow.source.name': "@remote\ndef train(...)",
                'mlflow.source.type': 'JOB',
            }
        )

        mlflow.sklearn.log_model(clf, "model", signature=signature)

In [ ]:
train_data = pd.read_csv('./data/iris.csv', header=None, engine="python")

이제 `train` 함수의 실행은 클라우드에서 이루어지며 SageMaker SDK가 데이터/변수의 직렬화/역직렬화 및 마샬링/언마샬링을 처리합니다.
모든 관련 파일은 패키징되어 SageMaker가 예상하는 방식으로 훈련 작업에서 사용할 수 있게 됩니다.

In [ ]:
train(train_data, 2)

In [ ]:
# get the last run in MLflow
last_run_id = mlflow.search_runs(
    experiment_ids=[experiment_id], 
    max_results=1, 
    order_by=["attributes.start_time DESC"]
)['run_id'][0]

mlflow_run_link = f"{presigned_url.split('/auth')[0]}/#/experiments/{experiment_id}/runs/{last_run_id}"

In [ ]:
display(Javascript('window.open("{}");'.format(mlflow_run_link)))

## 로컬 모드에서 SageMaker 훈련 작업 실행하기

우리는 또한 자체 관리형 컨테이너를 사용하고 스크립트만 전달하는 스크립트 모드에서 SageMaker를 사용할 수 있습니다.
먼저 모든 의존성이 이미 설치되어 있는지 확인합시다. 먼저 SageMaker AI Studio Domain에서 Docker가 활성화되어 있는지 확인합니다

In [ ]:
# check that docker enabled in the SageMaker domain
docker_settings = sm_client.describe_domain(DomainId=domain_id)['DomainSettings'].get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"The docker access is ENABLED in the domain {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"You must enable docker access in the domain to use Studio local mode")

그리고 `docker`가 설치되어 있는지 확인합니다.

In [ ]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Currently only Docker version 20.10.X is supported in Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validate the Docker Client is able to access Docker Server at [unix:///docker/proxy.sock]
docker version


이제 훈련 데이터를 처리하기 위한 `train.py` 스크립트를 작성해 보겠습니다.

In [ ]:
%%writefile training_code/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn import tree

import mlflow
from mlflow.models import infer_signature

mode = os.environ.get("MODE")
if mode is None:
    run_name = "Training"
else:
    run_name = "Local-Training"

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.
    parser.add_argument('--max_leaf_nodes', type=int, default=-1)

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])

    args = parser.parse_args()

    # Take the set of files and read them all into a single pandas dataframe
    input_files = [ os.path.join(args.train, file) for file in os.listdir(args.train) if os.path.isfile(os.path.join(args.train, file))]
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(args.train, "train"))
    raw_data = [ pd.read_csv(file, header=None, engine="python") for file in input_files ]
    train_data = pd.concat(raw_data)
    
    # Enable autologging in MLflow for SKlearn
    mlflow.sklearn.autolog()

    with mlflow.start_run(run_name=run_name) as run:
        # labels are in the first column
        train_y = train_data.iloc[:, 0]
        train_X = train_data.iloc[:, 1:]
    
        # Here we support a single hyperparameter, 'max_leaf_nodes'. Note that you can add as many
        # as your training my require in the ArgumentParser above.
        max_leaf_nodes = args.max_leaf_nodes
    
        # Now use scikit-learn's decision tree classifier to train the model.
        clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
        clf = clf.fit(train_X, train_y)
    
        predictions = clf.predict(train_X)
        signature = infer_signature(train_X, predictions)

        mlflow.set_tags(
            {
                'mlflow.source.name': "training_code/train.py",
                'mlflow.source.type': 'JOB',
            }
        )
    
        mlflow.sklearn.log_model(clf, "model", signature=signature)


## SageMaker 로컬 모델

로컬 모드에서 실행하고 MLflow 추적 서버에 로그 기록하기

In [ ]:
from sagemaker.local import LocalSession

LOCAL_SESSION = LocalSession()
LOCAL_SESSION.config = {'local': {'local_code': True}}  # Ensure full code locality, see: https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode


sklearn_local = SKLearn(
    entry_point="train.py",
    source_dir="training_code",
    framework_version="1.2-1",
    instance_type="ml.c5.xlarge",
    role=role,
    sagemaker_session=LOCAL_SESSION,
    hyperparameters={"max_leaf_nodes": 30},
    keep_alive_period_in_seconds=3600,
    environment={
        "MLFLOW_TRACKING_URI": mlflow_server_arn,
        "MODE": "local-mode",
        "LOGNAME": user_profile_name,
        "MLFLOW_EXPERIMENT_NAME": mlflow_experiment_name
    },
)

sklearn_local.fit({"train": train_input})

In [ ]:
experiment_id = mlflow.get_experiment_by_name(mlflow_experiment_name).experiment_id
# get the last run in MLflow
last_run_id = mlflow.search_runs(
    experiment_ids=[experiment_id], 
    max_results=1, 
    order_by=["attributes.start_time DESC"]
)['run_id'][0]


mlflow_run_link = f"{presigned_url.split('/auth')[0]}/#/experiments/{experiment_id}/runs/{last_run_id}"

In [ ]:
display(Javascript('window.open("{}");'.format(mlflow_run_link)))

관리형 인프라 모드에서 실행하고 MLflow 추적 서버에 로그 기록하기

In [ ]:
sklearn = SKLearn(
    entry_point="train.py",
    source_dir="training_code",
    framework_version="1.2-1",
    instance_type="ml.c5.xlarge",
    role=role,
    sagemaker_session=sagemaker_session,
    hyperparameters={"max_leaf_nodes": 30},
    keep_alive_period_in_seconds=3600,
    environment={
        "MLFLOW_TRACKING_URI": mlflow_server_arn,
        "LOGNAME": user_profile_name,
        "MLFLOW_EXPERIMENT_NAME": mlflow_experiment_name
    },
)

sklearn.fit({"train": train_input})

In [ ]:
# get the last run in MLflow
last_run_id = mlflow.search_runs(
    experiment_ids=[experiment_id], 
    max_results=1, 
    order_by=["attributes.start_time DESC"]
)['run_id'][0]

mlflow_run_link = f"{presigned_url.split('/auth')[0]}/#/experiments/{experiment_id}/runs/{last_run_id}"

In [ ]:
display(Javascript('window.open("{}");'.format(mlflow_run_link)))

## MLflow 모델 등록하기

In [ ]:
registered_model_name = "sm-immersion-day-model"

# construct the model URI
model_uri = f"runs:/{last_run_id}/model"

# register the model
registered_model_version = mlflow.register_model(model_uri, registered_model_name)

In [ ]:
# get SageMaker model registry data for this model version
model_package_group_name = sm_client.list_model_package_groups(NameContains=registered_model_name)['ModelPackageGroupSummaryList'][0]['ModelPackageGroupName']
sm_model_package = sm_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name,
        SortBy="CreationTime",
        SortOrder="Descending",
    )['ModelPackageSummaryList'][0]



In [ ]:
sm_model_package

In [ ]:
model_approval_status = 'PendingManualApproval'

# update SageMaker model version with mlflow cross-reference
sm_client.update_model_package(
        ModelPackageArn=sm_model_package['ModelPackageArn'],
        ModelApprovalStatus=model_approval_status,
        ApprovalDescription="created a new model version",
        CustomerMetadataProperties={
            "mlflow_model_name": registered_model_version.name,
            "mlflow_model_uri": model_uri,
            "mlflow_experiment_name": mlflow_experiment_name,
        },
)


In [ ]:


# Show the model registry link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}/versions">the model package group</a> in the Studio UI</b>'.format(
            domain_id, region, model_package_group_name))
)

